In [ ]:
import numpy as np

import sklearn
from sklearn.datasets import make_blobs
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml, fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint

from sklearn.metrics import mean_squared_error

from sklearn.datasets import fetch_covtype

import torch

from torch.utils.data import TensorDataset, DataLoader

from torchvision.transforms.functional import normalize
import torch.nn.functional as F

In [ ]:
torch.cuda.is_available()

### Tensors

In [ ]:
x = torch.tensor([5.5, 3])
print(x)

x = torch.zeros(2, 4)
print(x)

x = torch.rand(5, 3)
print(x)

In [ ]:
y = torch.ones(5,3)
z = x+y
print(z)

In [ ]:
a = torch.rand(1)
print(a)
print(a.item())

In [ ]:
b = z.numpy()
print(b)

c = torch.from_numpy(b)
print(c)

### Automatic Differentiation

#### Scalar derivatives

In [ ]:
a = torch.tensor(1., requires_grad = True)
print(a)

In [ ]:
b = torch.exp(a**2*torch.cos(a))
print(b)

In [ ]:
b.backward()

In [ ]:
print(a.grad)

In [ ]:
aa = 1.
print(-aa * np.exp(aa**2 * np.cos(aa)) * (aa * np.sin(aa) - 2 * np.cos(aa)))

In [ ]:
aten = torch.rand(2,2)
print(aten)
print(aten*aten)
print(aten[0,0]*aten[0,0])
print(aten.mm(aten))

In [ ]:
aten = torch.rand(2,2,requires_grad = True)
print(aten)
bten = torch.exp(aten**2*torch.cos(aten))
bten.backward(torch.Tensor([[1.,1.],[1.,1.]]))
print(aten.grad)

In [ ]:
aaten = aten[1,0].item()
print(-aaten * np.exp(aaten**2 * np.cos(aaten)) * (aaten * np.sin(aaten) - 2 * np.cos(aaten)))

#### Jacobians

In [ ]:
a = torch.tensor([1.], requires_grad = True)
b = torch.tensor([2.], requires_grad = True)


In [ ]:
c1 = a**2*b
c2 = b*torch.cos(a)


In [ ]:
c = torch.cat((c1,c2))
print(c)


In [ ]:

for col in ([1.,0.],[0.,1.]):
    c.backward(torch.tensor(col), retain_graph = True)
    print(a.grad)
    print(b.grad)
    a.grad.data.zero_() # Otherwise the next pass sums with the previous one
    b.grad.data.zero_()

In [ ]:
aa = 1.
bb = 2.
dc1daa = 2*aa*bb
dc1dbb = aa**2
dc2daa = -bb*np.sin(aa)
dc2dbb = np.cos(aa)
print([[dc1daa,dc1dbb],[dc2daa,dc2dbb]])

#### Logistic Regression from Scikit-Learn

In [ ]:
Xb,yb = make_blobs(n_samples = 250, n_features = 2,
                   centers = [[1.3,1],[0.5,2.4]],
                  cluster_std = 0.45, shuffle = True, random_state = 42)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=5984,penalty='none').fit(Xb, yb)
print(clf.coef_)
print(clf.intercept_)

In [ ]:
ypred = clf.predict_proba(Xb)[:,1]
print(ypred)

In [ ]:
from sklearn.metrics import log_loss

log_loss(yb,ypred)

In [ ]:
h = .02
x_min, x_max = Xb[:, 0].min() - 1, Xb[:, 0].max() + 1
y_min, y_max = Xb[:, 1].min() - 1, Xb[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

from matplotlib.colors import ListedColormap
cmap_light = ListedColormap(['cyan', 'orange', 'lightgreen'])

Z = Z.reshape(xx.shape)

plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light,shading='auto')

plt.plot(Xb[:, 0][yb==0], Xb[:, 1][yb==0], "bo")
plt.plot(Xb[:, 0][yb==1], Xb[:, 1][yb==1], "ro")

plt.axis('tight')

#### Custom-written single layer neural network version of logistic regression

In [ ]:
X = torch.from_numpy(Xb)
y = torch.from_numpy(yb).unsqueeze(0).T
print(y)


#### Show torch-based loss calculation on sklearn's results

In [ ]:
ws = torch.from_numpy(clf.coef_.T)
bs = torch.from_numpy(clf.intercept_).unsqueeze(0)
y_s = 1./(1. + torch.exp(-(X.mm(ws)+bs)))
loss_s = (- y * torch.log(y_s) - (1.-y) * torch.log(1. - y_s)).mean()
loss_elements = (- y * torch.log(y_s) - (1.-y) * torch.log(1. - y_s))
print(loss_s)

#### Implement gradient descent using torch's autodiff

In [ ]:
indim = 2
outdim = 1
dtype = torch.double

w = torch.randn(indim,outdim,dtype=dtype,requires_grad=True)
b = torch.randn(1,outdim,dtype=dtype,requires_grad=True)
learning_rate = 5e-2

for t in range(50000):
    y_pred = 1./(1. + torch.exp(-(X.mm(w)+b)))
    loss = (- y * torch.log(y_pred) - (1.-y) * torch.log(1. - y_pred)).mean()
    
    if t % 1000 == 999:
        print("Step {}: Loss = {}".format(t,loss.item()))
    
    loss.backward()
    
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
        
        w.grad.zero_()
        b.grad.zero_()

print(w)
print(b)

In [ ]:
ws = w
bs = b
y_s = 1./(1. + torch.exp(-(X.mm(ws)+bs)))
loss_s = (- y * torch.log(y_s) - (1.-y) * torch.log(1. - y_s)).mean()
loss_elements = (- y * torch.log(y_s) - (1.-y) * torch.log(1. - y_s))
print(loss_s)

In [ ]:
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
h = .02  # step size in the mesh
x_min, x_max = Xb[:, 0].min() - 1, Xb[:, 0].max() + 1
y_min, y_max = Xb[:, 1].min() - 1, Xb[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# Flatten the grid points, align them as columns, and predict them
Xgrid = torch.from_numpy(np.c_[xx.ravel(), yy.ravel()])
y_round = (1./(1. + torch.exp(-(Xgrid.mm(w)+b)))).round()

# Create color maps
from matplotlib.colors import ListedColormap
cmap_light = ListedColormap(['cyan', 'orange', 'lightgreen'])

# Put the result into a color plot
Z = y_round.reshape(xx.shape).detach().numpy()

plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light,shading='auto')

plt.plot(Xb[:, 0][yb==0], Xb[:, 1][yb==0], "bo")
plt.plot(Xb[:, 0][yb==1], Xb[:, 1][yb==1], "ro")

plt.axis('tight')

#### Logistic regression using some built-in models

In [ ]:
indim = 2
outdim = 1
X = torch.from_numpy(Xb).float()
y = torch.from_numpy(yb).unsqueeze(0).T.float()

logreg = torch.nn.Sequential(
    torch.nn.Linear(indim,outdim,bias=True),
    torch.nn.Sigmoid()
)
loss_fn = torch.nn.BCELoss()
learning_rate = 5e-2
optimizer = torch.optim.SGD(logreg.parameters(), lr=learning_rate)

for t in range(50000):
    y_pred = logreg(X)
    loss = loss_fn(y_pred,y)
    
    if t % 1000 == 999:
        print("Step {}: Loss = {}".format(t,loss.item()))
    
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()

In [ ]:
w = logreg[0].weight.data.T
b = logreg[0].bias.data.T
ws = w
bs = b
y_s = 1./(1. + torch.exp(-(X.mm(ws)+bs)))
loss_s = (- y * torch.log(y_s) - (1.-y) * torch.log(1. - y_s)).mean()
loss_elements = (- y * torch.log(y_s) - (1.-y) * torch.log(1. - y_s))
print(loss_s)

# Dense Multilayer Neural Network Examples

## Regression on the California housing set

In [1]:
X_reg, y_reg = fetch_california_housing(return_X_y=True)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size = 0.2,
                                    shuffle = True, random_state = 42)

scaler = StandardScaler()
scaler.fit(X_train_reg)
X_train_reg = scaler.transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)

NameError: name 'fetch_california_housing' is not defined

In [ ]:
X_train = torch.from_numpy(X_train_reg).float()
X_test = torch.from_numpy(X_test_reg).float()
y_train = torch.from_numpy(y_train_reg).float().unsqueeze(1)
y_test = torch.from_numpy(y_test_reg).float().unsqueeze(1)

In [ ]:
print(y_train)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

bs = 32
train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)
test_ds = TensorDataset(X_test, y_test)
test_dl = DataLoader(test_ds, batch_size=len(test_ds))


In [ ]:
indim = 8
hiddim = 10
outdim = 1

learning_rate = 2e-4
epochs = 150

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(indim,hiddim,bias=True),
    torch.nn.ReLU(),
    # torch.nn.Linear(hiddim,hiddim,bias=True),
    # torch.nn.ReLU(),
    # torch.nn.Linear(hiddim,hiddim,bias=True),
    # torch.nn.ReLU(),
    torch.nn.Linear(hiddim,outdim,bias=True),
)

optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

loss_fn = torch.nn.MSELoss() # For regression

### Training loop

In [ ]:
size = len(train_dl.dataset)
for epoch in range(epochs):
    for batch, (X, y) in enumerate(train_dl):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch % 10 == 0) and (batch == 0):
            print("Epoch {} Batch {}: Training Loss = {}".format(epoch,batch,
                                                        loss.item()))

### Test loop

In [ ]:
with torch.no_grad():
    for X, y in test_dl:
        pred = model(X)
        loss_value = loss_fn(pred, y).item()
        print("Test Loss = {}".format(loss_value))

## Classification on the forest covertype dataset

In [ ]:
X_clf, y_clf = fetch_covtype(return_X_y=True)
y_clf = y_clf - 1 # Need to change the classification categories to be indexed at 0
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size = 0.2,
                                    shuffle = True, random_state = 42)

scaler = StandardScaler()
scaler.fit(X_train_clf)
X_train_clf = scaler.transform(X_train_clf)
X_test_clf = scaler.transform(X_test_clf)

In [ ]:
X_train = torch.from_numpy(X_train_clf).float()
X_test = torch.from_numpy(X_test_clf).float()
y_train = torch.from_numpy(y_train_clf).long()#.unsqueeze(1)
y_test = torch.from_numpy(y_test_clf).long()#.unsqueeze(1)

In [ ]:
print(y_train)

In [ ]:
bs = 32
train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)
test_ds = TensorDataset(X_test, y_test)
test_dl = DataLoader(test_ds, batch_size=len(test_ds))

indim = 54
hiddim = 20
hiddim2 = 20
outdim = 7

learning_rate = 1e-3
epochs = 10

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(indim,hiddim,bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(hiddim,hiddim2,bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(hiddim,outdim,bias=True),
)

optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

loss_fn = torch.nn.CrossEntropyLoss()

## Training loop

In [ ]:
size = len(train_dl.dataset)
for epoch in range(epochs):
    for batch, (X, y) in enumerate(train_dl):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        y_pred = torch.argmax(pred.data,1)
        correct = (y == y_pred).sum().item()/len(y_pred)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch % 1 == 0) and (batch % 1000 == 0):
            print("Epoch {} Batch {}: Training Loss = {}: Accuracy = {}".format(epoch,batch,
                                                        loss.item(),correct))

### Test loop

In [ ]:
with torch.no_grad():
    for X, y in test_dl:
        pred = model(X)
        loss = loss_fn(pred, y)
        loss_value = loss.item()
        
        y_pred = torch.argmax(pred.data,1)
        correct = (y == y_pred).sum().item()/len(y_pred)

        print("Test Loss = {}: Accuracy = {}".format(loss_value,correct))

# Convolutional Neural Network classification on the MNIST dataset

In [ ]:
mnist = fetch_openml('mnist_784', version=1, cache=True)
mnist.target = mnist.target.astype(np.int8)
X_cnn, y_cnn = mnist["data"], mnist["target"]
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_cnn, test_size = 0.2,
                                    shuffle = True, random_state = 42)


In [ ]:
X_train_cnn = X_train_cnn/255.
X_test_cnn = X_test_cnn/255.

X_trainf = torch.from_numpy(X_train_cnn.to_numpy()).float()
X_testf = torch.from_numpy(X_test_cnn.to_numpy()).float()
y_train = torch.from_numpy(y_train_cnn.to_numpy()).long()
y_test = torch.from_numpy(y_test_cnn.to_numpy()).long()

X_train = X_trainf.view(-1,1,28,28)
X_test = X_testf.view(-1,1,28,28)
plt.imshow(X_train[0][0], cmap='gray')
print(y_train[0])

In [ ]:
conv1 = torch.nn.Conv2d(1,10,5)
pool = torch.nn.MaxPool2d(2)
conv2 = torch.nn.Conv2d(10,20,5)

In [ ]:
print(X_train[0:1].shape)
print(conv1(X_train[0:1]).shape)
print(pool(conv1(X_train[0:1])).shape)
print(F.relu(pool(conv1(X_train[0:1]))).shape)
print(conv2(F.relu(pool(conv1(X_train[0:1])))).shape)
print(pool(conv2(F.relu(pool(conv1(X_train[0:1]))))).shape)

imgout = F.relu(pool(conv2(F.relu(pool(conv1(X_train[0:1]))))))
print(imgout.shape)
print(np.prod(imgout.shape))


In [ ]:
bs = 32
train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)
test_ds = TensorDataset(X_test, y_test)
test_dl = DataLoader(test_ds, batch_size=len(test_ds))
learning_rate = 1e-3
epochs = 1

class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = torch.nn.Conv2d(1,10,5)
        self.pool = torch.nn.MaxPool2d(2)
        self.conv2 = torch.nn.Conv2d(10,20,5)
        self.fc1 = torch.nn.Linear(320,50)
        self.fc2 = torch.nn.Linear(50,10)
    def forward(self,x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = CNN()

optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
size = len(train_dl.dataset)
for epoch in range(epochs):
    for batch, (X, y) in enumerate(train_dl):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        y_pred = torch.argmax(pred.data,1)
        correct = (y == y_pred).sum().item()/len(y_pred)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch % 1 == 0) and (batch % 100 == 0):
            print("Epoch {} Batch {}: Training Loss = {}: Accuracy = {}".format(epoch,batch,
                                                        loss.item(),correct))

In [ ]:
with torch.no_grad():
    for X, y in test_dl:
        pred = model(X)
        loss = loss_fn(pred, y)
        loss_value = loss.item()
        
        y_pred = torch.argmax(pred.data,1)
        correct = (y == y_pred).sum().item()/len(y_pred)

        print("Test Loss = {}: Accuracy = {}".format(loss_value,correct))